In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul  3 04:44:44 2018

@author: ho
"""

# -*- coding: utf-8 -*-
"""
Created on Mon Jun 25 11:08:38 2018

@author: ho
"""

## 50.021 Artificial Intelligence HW6
## Done By: Ho Hui Qi 1001612
## Run code using python v3.6.3, coded in Spyder IDE, Windows

import torch
import torch.nn as nn
import string
import random
import matplotlib.pyplot as plt
import copy

# =============================================================================
# LOAD DATA INTO DICT
# =============================================================================
def getInput(filepath):  
    '''Takes in a filepath to the csv file with lines from "star trek"
    and splits into lines, then returns a dictionary with 'star trek' as key
    and a list of lines as value'''
    # create an empty dictionary
    category_lines = {}
    # create an empty list to store the lines
    lines = []
    # define 'filterowrds'
    filterwords = ['NEXTEPISODE']
    # open csv file with star trek lines
    with open(filepath,'r') as f:
        for line in f:
#            print (line)
            # remove the spaces and replace symbols with spaces
            v1 = line.strip().replace('=','').replace('/',' ').replace('+',' ').replace('(',' ')
            v2 = v1.replace('[',' ').replace(')',' ').replace(']',' ').split(',')
#            print (v2)
            # for each line (list)
            for w in v2:
                # check that it is not filterwords and is longer than 1
                # meaning it is legit line
                if (w not in filterwords) and (len(w)>1):
                    if w[0] == ' ':
                        w = w[1:]
                    # append the line to our list 
                    lines.append(w)
    # add the list to our dictionary with 'star trek' as key
    category_lines['star trek'] = lines
    # return the dictionary
    return category_lines

# =============================================================================
# CREATE THE LSTM NETWORK
# =============================================================================
class RNNLSTM(nn.Module):
    # add dropout probability
    def __init__(self, inp_size, hidden_dim, num_letters, n_layers, dropout_prob):
        super(RNNLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.num_letters = num_letters
        self.lstm = nn.LSTM(input_size=inp_size, hidden_size=hidden_dim, num_layers=n_layers, dropout=dropout_prob)
        self.fc = nn.Linear(in_features=hidden_dim, out_features=num_letters)
        self.ls = torch.nn.LogSoftmax(dim=1)
    
    # no need batchsize
    def forward(self, x0, h0, c0, debug = False):
        # x0 is a list of tensors 
        x = torch.nn.utils.rnn.pack_sequence(x0) 
        # get the output, hidden state and memory of the nlayer lstm
        output, (hn, cn) = self.lstm(x, (h0, c0))
        # get the hidden state from the second LSTM layer
        preout = hn[self.n_layers-1,:,:]
        # run preout through fully connected layer and get soft max
        # for NLL loss, use log softmax
        z = self.ls(self.fc(preout))
        return z
        
# =============================================================================
# CONVERT TO TENSORS
# =============================================================================
'''
# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor
'''

all_letters = string.ascii_letters + " .,;'-:?!" + "0123456789" 
#print (all_letters[52])
n_letters = len(all_letters) + 1 # Plus EOS marker

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    '''Takes in a string (line of letters) and converts into tensor
    with dimensions [length of line, 1, total number of letters]; 
    inputs into our LSTM network'''
    tensor = torch.zeros(len(line),1,n_letters)
    for letter_idx in range(len(line)):
        letter = line[letter_idx]
        tensor[letter_idx][0][all_letters.find(letter)] = 1
    # add the EOS token
#    tensor[len(line)][0][n_letters-1] = 1
    
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    '''Takes in a string (line of letters) and converts into tensor
    with dimensions [length of line]; targets or true labels'''
    letter_indexes = [all_letters.find(letter) for letter in line[1:]]
#    print (letter_indexes)
    letter_indexes.append(n_letters-1) # EOS
#    print(len(letter_indexes))
    return torch.LongTensor(letter_indexes)

# =============================================================================
# SPLIT DATA INTO TRAINING AND TESTING
# =============================================================================
# Use 80% for training and 20% for testing
# Temperatured sampling?
    
# split data based on trainratio
def dataSpliter(data, trainratio):
    '''Takes in a list of lines from dictionary 'data' (with key 'st')
    and splits the list based on 'trainratio', returns 2 lists, one with
    the lines for training and another for testing'''
    trainlist = []
    testlist = []
    # list of lines from data (dictionary) 
    linelist = data['star trek']
    # shuffle the list
    random.shuffle(linelist)
    cut = int(trainratio*len(linelist))
    trainlist = linelist[0:cut]
    testlist = linelist[cut:]
    return trainlist, testlist

#print (trainlist)
#print (testlist)
'''
traintensors = []
for line in trainlist:
    t = inputTensor(line)
    traintensors.append(t)

testtensors = []
for line in testlist:
    t = inputTensor(line)
    testtensors.append(t)

print (traintensors)
print (testtensors)
'''

def sampleletter():
    letters = string.ascii_uppercase
    letters = letters.replace('Q','').replace('X','').replace('Y','')
    idx = random.randint(0,len(letters)-1)
#    print (letters)
    return letters[idx]

def main(hdim, nlayer, epochs):
    # Training settings
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # extract data from star trek file and store in datadict with 'st' as key
    datadict = getInput('./star_trek_transcripts_all_episodes_f.csv')
    
    # split 'st' data set into training and testing dataset 
    trainlist, testlist = dataSpliter(datadict, trainratio=0.8)
    
    # initialize lstm model
    lstm = RNNLSTM(inp_size=n_letters, hidden_dim=hdim, num_letters=n_letters, 
                   n_layers=nlayer, dropout_prob=0.1).to(device)
    
    # define the loss function for backpropagation
    loss_function = nn.NLLLoss()
    
    print ('Predicting Star Trek lines \n')
    print ('LSTM with', hdim, 'hidden dimensions &', nlayer, 'layers')
    print ('------------------------------------')
    
    # set learning rate
    learning_rate = 0.005
    
    # intialise empty lists to store losses
    train_loss = []
    test_loss = []
    test_acc = []
    
    # define a best acc value to compare against
    bestacc = 0
    
    for ep in range(epochs):   
        print ('epoch', ep, 'of', epochs, ':')
        
        # =====================================================================
        # TRAINING PHASE
        # =====================================================================
        print (' TRAINING...')
        avgtrainloss = 0
        
        for line in trainlist:
#            print (line)
            # convert input line to tensor of characters
            inp = inputTensor(line).to(device)
            # convert input line to tensor of indexes
            targ = targetTensor(line).unsqueeze(-1).to(device)
            
            # initlialize hidden laeyrs and cell state
            h0 = torch.zeros(nlayer,1,hdim)
            c0 = torch.zeros(nlayer,1,hdim)
            
            # set lstm to train mode
            lstm.train()
            
            # set lstm to zero
            lstm.zero_grad()
            
            # initialise losses
            losses = 0
            avgloss = 0
        
            for i in range(inp.shape[0]):
                # pass input into the lstm
                output = lstm(inp[i].unsqueeze(0), h0, c0)
                # keep a copy of the output to prevent aliasing
                outputcopy = output.clone()
                # calculate the loss with NLL loss
                ls = loss_function(outputcopy,targ[i])
                # add the losses for each line
                losses += ls
                # total losses per line
                avgloss += ls.item()
            # average loss for each line
            avgtrainloss += avgloss/len(line)
            # backpropagate over losses
            losses.backward()

            # update parameters            
            for p in lstm.parameters():
                p.data.add_(-learning_rate, p.grad.data)
        trainloss = avgtrainloss/len(trainlist)
        print('Train avgloss', trainloss)
        # add to the train_loss list for each epoch
        train_loss.append(trainloss)

        # =====================================================================
        # SAMPLING PHASE (TEMPERATURED)
        # =====================================================================
        print ('\n SAMPLING...')
        # define a max length of sequence to output
        max_length = 20
        # define the number of samples we want
        n_samples = 10
        # define temperature
        temp = 0.5
        
        for s in range(n_samples):
            # randomly pick a starting letter
            start_letter = sampleletter()

            with torch.no_grad():
                # convert start letter to input tensor
                test = inputTensor(start_letter)
                # add start letter to output line
                outputline = start_letter
                
                # initialise hidden layers and cell state
                h0 = torch.zeros(nlayer,1,hdim)
                c0 = torch.zeros(nlayer,1,hdim)
                
                # set lstm to eval mode
                lstm.eval()
                
                # for each character in the sample line
                for i in range(max_length):
                    # pass input through lstm
                    output = lstm(test, h0, c0)
                    # get temperatured output, softmax it
                    tempout = torch.nn.functional.softmax(output/temp, dim=1)
                    # get a sample from the multinomial dist
                    samplei = torch.multinomial(tempout,1,replacement=False)
                    # find the index of the sampled letter
                    samplei = samplei[0][0]
                    
                    if samplei == n_letters - 1: #EOS
                        break
                    else:
                        # retrieve the best letter
                        letter = all_letters[samplei]
                        # append letter to list 
                        outputline += letter
                    # use the best letter as the input 
                    test = inputTensor(letter)
                    
                print (outputline)
            
        # =====================================================================
        # TESTING PHASE
        # =====================================================================
        print ('\n TESTING...')
        
        with torch.no_grad():
            avgtestloss = 0
            avgtestacc = 0
            
            # for each line in the test set
            for line in testlist:
                '''
                outputline = line[0]
                '''
                # convert line to input tensors
                test = inputTensor(line).to(device)
                # convert line to target tensor
                targ = targetTensor(line).unsqueeze(-1).to(device)

                # initialise hidden layers and cell state
                h0 = torch.zeros(nlayer,1,hdim)
                c0 = torch.zeros(nlayer,1,hdim)
                
                # initialise losses
                losses = 0
                testacc = 0
                
                # set lstm to eval mode
                lstm.eval()
                
                # for each character in the line
                for i in range(len(line)):
                    # pass character through the lstm
                    output = lstm(test[i].unsqueeze(0), h0, c0)
                    # copy the output to prevent aliasing
                    outputcopy = output.clone()
                    # find the loss using loss function
                    ls = loss_function(outputcopy,targ[i])
                    # find the best letter prediction
                    topv, topi = outputcopy.topk(1)
                    # get the index of the best prediction
                    topi = topi[0][0]
                    # check if the predcition matches the actual letter
                    testacc += (topi==targ[i]).item()
                    # sum losses over each line
                    losses += ls.item()
                    
                    if topi == n_letters-1: #EOS
                        break
                    else:
                        # find the letter corresponding to the best index
                        letter = all_letters[topi]
                        '''
                        outputline += letter
                        '''
                '''
                print (line, '//', outputline)
                '''
                # average accuracy over each line
                avgtestacc += testacc/len(outputline)
                # average losses over each line
                avgtestloss += losses/len(outputline)
                
        testloss = avgtestloss/len(testlist)
        testacc = avgtestacc/len(testlist)
        
        # add average test loss per eopch to list
        test_loss.append(testloss)
        print('Test avgloss', testloss)
        # add average test accuracy per eopch to list
        test_acc.append(testacc*100)
        print('Test accuracy', testacc*100, '%')
        print ('\n')
        
        # keep track of the weights that give the best test accuracy
        if testacc > bestacc:
            bestacc = testacc
            best_model_wts = copy.deepcopy(lstm.state_dict())
    
    # save this model in a file named 'model.pt'
    torch.save(best_model_wts,'./model.pt')     
    
    # =========================================================================
    # PLOTS
    # =========================================================================
    x = list(range(epochs))
    
    ax = plt.subplot(311)
    # plot train and test error on the same graph
    ax.plot(x, train_loss, 'r', label='train error')
    ax.plot(x, test_loss, 'b', label='test error')
    ax.set_title('Errors with epochs')
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Errors')        
    
    ax.legend()
    
    bx = plt.subplot(313)
    # plot test accuracy on a separate graph
    bx.plot(x, test_acc, 'b', label='test accuracy')
    bx.set_title('Test accuracy with epochs')
    bx.set_xlabel('Epochs')
    bx.set_ylabel('Test Accuracy') 

    # show the plot
    plt.show()


main(hdim=200, nlayer=2, epochs=10)

In [ ]:
torch.cuda.is_available()